In [1]:
#Import the dependencies / Inspect the Data
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
data = 'sample_py.csv'
data = pd.read_csv(data)
data = pd.DataFrame(data)
list(data.columns)

['Order number',
 'Customer',
 'Country',
 '  Revenue amount  ',
 'Date',
 'Unnamed: 5']

In [2]:
#Clean the Data
data.drop(['Unnamed: 5'], axis = 1, inplace = True)
data = data.reset_index(drop = True)
data

,Order number,Customer,Country,Revenue amount,Date
0,1,E,Italy,$859,1/1/2020
1,2,N,China,$848,1/1/2020
2,3,Z,Italy,"$1,038",1/1/2020
3,4,W,India,"$2,321",1/1/2020
4,5,N,United States,"$1,497",1/3/2020
...,...,...,...,...,...
2019,2020,O,Sweden,"$2,025",12/31/2020
2020,2021,Q,Brazil,"$1,151",12/31/2020
2021,2022,C,Canada,"$2,921",12/31/2020
2022,2023,C,Sweden,"$2,364",12/31/2020


In [11]:
#More Wrangling so that the data can easily be grouped by the Month
data['Date'] = pd.to_datetime(data['Date'])
# data['Revenue amount'] = data['Revenue amount'].str.replace(',', '')
# data['Revenue amount'] = data['Revenue amount'].str.replace('$', '')
# data['Revenue amount'].astype(int)
data['Month'] = data['Date'].dt.month
# sam_sorted
data

,Order number,Customer,Country,Revenue amount,Date,Month
0,1,E,Italy,$859,2020-01-01,1
1,2,N,China,$848,2020-01-01,1
2,3,Z,Italy,"$1,038",2020-01-01,1
3,4,W,India,"$2,321",2020-01-01,1
4,5,N,United States,"$1,497",2020-01-03,1
...,...,...,...,...,...,...
2019,2020,O,Sweden,"$2,025",2020-12-31,12
2020,2021,Q,Brazil,"$1,151",2020-12-31,12
2021,2022,C,Canada,"$2,921",2020-12-31,12
2022,2023,C,Sweden,"$2,364",2020-12-31,12


In [13]:
#Figure out the amount of New/Returning Customers per Month
#The key is to obtain the UNIQUE 'Customer Name' Values by Month and then compare the difference between the unique value
#

schools_list = []
new_schools = []

months = sorted(list(data['Month'].unique()))
for m in months:
    recs_df = data.loc[data['Month'] == m]
    
    schools_month = list(recs_df['Customer'].unique())

    diff = set(schools_month) - set(schools_list)
    
    schools_list = schools_list + schools_month
    
    new_schools_month = {
        'Month': m,
        'Schools': len(schools_month),
        'Count of New Schools': len(diff)
    }
    
    new_schools.append(new_schools_month)

    

In [14]:
#Now Create the DataFrame based on the calculations from the Loop
new_schools_df = pd.DataFrame(new_schools)
new_schools_df['Count of New Schools'].sum()
new_schools_df['Count of Returning Schools'] = new_schools_df['Schools'] - new_schools_df['Count of New Schools']
new_schools_df

,Month,Schools,Count of New Schools,Count of Returning Schools
0,1,26,26,0
1,2,26,0,26
2,3,26,0,26
3,4,26,0,26
4,5,26,0,26
5,6,26,0,26
6,7,26,0,26
7,8,26,0,26
8,9,26,0,26
9,10,26,0,26


In [ ]:
#Grab Revenue by Month by grouping revenue by Month
revenues = sam_sorted[['Month', 'Revenue amount']].astype(int)
revenues = pd.DataFrame(revenues)
rev_group = revenues.groupby("Month").sum()
rev_group

In [ ]:
#Find the Error and look through most frequent customers - see excel
sam_sorted['Customer Name'].value_counts().head(20)

In [ ]:
#Build Annual Summary - #SEE BELOW FOR ACTUAL REPORT
total_revenue = sam_sorted['Revenue amount'].astype(int).sum()
transactions = sam_sorted['Order number'].count()
rev_per = total_revenue / transactions
most_rev = rev_group.max()
most_rev = most_rev['Revenue amount']
least_rev = rev_group.min()
least_rev = least_rev['Revenue amount']
avg_rev = rev_group.mean()
top_five = sam_sorted['Country'].value_counts().head()


snapshot = pd.DataFrame(
    {'2020 Revenue' : total_revenue,
     '2020 Transactions' : transactions,
     'Avg. Revenue per Transaction' : rev_per,
     'Most Profitable Month' : most_rev,
     'Least Profitable Month' : least_rev,
     'Average Monthly Revenue' : avg_rev,
        
    }
)

snapshot['2020 Revenue'] = snapshot['2020 Revenue'].map("${:,}".format)
snapshot['2020 Transactions'] = snapshot['2020 Transactions'].map("{:,}".format)
snapshot['Avg. Revenue per Transaction'] = snapshot['Avg. Revenue per Transaction'].map("${:,.2f}".format)
snapshot['Most Profitable Month'] = snapshot['Most Profitable Month'].map("${:,}".format)
snapshot['Least Profitable Month'] = snapshot['Least Profitable Month'].map("${:,}".format)
snapshot['Average Monthly Revenue'] = snapshot['Average Monthly Revenue'].map("${:,.2f}".format)


snapshot

In [ ]:
x = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
y = new_schools_df['Count of New Schools']
y2 = new_schools_df['Count of Returning Schools']
fig1 = make_subplots(specs=[[{"secondary_y": True}]])
fig1.add_trace(go.Scatter(x=x, y=y, name="New Clients"), secondary_y=False,)
fig1.add_trace(go.Scatter(x=x, y=y2, name="Repeat Clients"),secondary_y=True,)
fig1.update_layout(title_text="Customer Breakdown")
fig1.update_xaxes(title_text="<b>Month 2020<b>")
fig1.update_yaxes(title_text="<b>New</b> Clients", secondary_y=False)
fig1.update_yaxes(title_text="<b>Repeat</b> Clients", secondary_y=True)


In [ ]:
rev = rev_group['Revenue amount']
months_bar = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

fig2 = go.Figure()
fig2.add_trace(go.Bar(x=months_bar, y=rev, name='Revenue Amount',marker_color='blue'))
fig2.update_yaxes(title_text="<b>Revenue</b>")
fig2.update_layout(title_text="Revenue by Month - 2020")
fig2.update_layout(yaxis_range = [350000,425000])
fig2.update_layout(barmode='group', xaxis_tickangle=-45)
fig2.update_xaxes(title_text="<b>Month 2020<b>")
fig2.show()



In [ ]:
transactions = sam_sorted.groupby('Month').count()['Order number']
countries = sam_sorted.groupby('Country').count()['Order number']
countries = pd.DataFrame(countries).sort_values('Order number', ascending = False).head(10)

In [ ]:
values = list(countries['Order number'])
colors = ['lightred', 'lightblue', 'darkorange', 'lightgreen', 'indianred', 'lavander', 'mediumturquoise', 'gold']

fig3 = go.Figure(data=[go.Pie(labels=['United States','Italy','Australia','United Kingdom', 'China', 'Belgium', 'Singapore', 'Sweden', 'Canada', 'Brazil'], values= values)])
fig3.update_traces(hoverinfo = 'label+percent', textinfo = 'value', textfont_size = 18, marker = dict(colors=colors, line = dict(color='#000000', width=2)))
fig3.update_layout(title_text="Top Ten Countries by Transaction")

In [ ]:
#_______________________________________________________ANNUAL REPORT_____________________________________________________#
snapshot

In [ ]:
#______________________________________________________ANNUAL REPORT_______________________________________________________#
fig1.show()
fig2.show()
fig3.show()